In [178]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

# Load Clean Data

In [10]:
folder = '../data/clean_data/'

df_categories = pd.read_csv(f'{folder}df_categories.csv')
df_comments = pd.read_parquet(f'{folder}df_comments_video.parquet')
df_transcript = pd.read_csv(f'{folder}df_transcript_original.csv')
df_videos = pd.read_csv(f'{folder}df_video_data.csv')
df_channel = pd.read_csv(f'{folder}df_channel_data.csv')

# Vectorización Textos TF-IDF

# Modelo 1

Variables: `clean_text`, `video_id`

In [74]:
model1 = df_transcript[df_transcript['language'] == 'english'][['clean_text', 'video_id']].copy()
model1

,clean_text,video_id
0,music thank hello everyone hope great era ai w...,qtlUwwtvuEg
1,number three music facebook enacted emergency ...,QaoDXYYtgK0
2,series going introduce deep learning least per...,PqDwddEHswU
3,earn money ai part two let go want create kind...,B-Y7rnOa43w
4,current medical science run option doctor dont...,vyit-1zKsZ4
...,...,...
1781,good afternoon started recording right welcome...,uuh7spVdf0c
1782,machine learning big data changing econometric...,Bm6CAjVtrIw
1783,music applause thanks much hadley thank coming...,atiYXm7JZv0
1785,happened last year made urgent long time worki...,ZQazWxegNm8



El parámetro ngram_range en TfidfVectorizer se refiere a los tamaños de n-gramas que el vectorizador debe considerar al analizar el texto. Un n-grama es una secuencia de "n" palabras consecutivas en un texto.

Desglose de ngram_range=(1, 2):
Unigrama (1-gram): Es una secuencia de 1 palabra. Si ngram_range=(1, 1), el vectorizador solo consideraría palabras individuales.

Ejemplo: Para la frase "hola mundo", los unigramas serían: ["hola", "mundo"].
Bigramas (2-gram): Es una secuencia de 2 palabras consecutivas. Si ngram_range=(2, 2), el vectorizador solo consideraría pares de palabras consecutivas.

Ejemplo: Para la frase "hola mundo", los bigramas serían: ["hola mundo"].
ngram_range=(1, 2): Con este rango, el vectorizador considerará tanto unigramas como bigramas. Es decir, se generarán características tanto para palabras individuales como para pares de palabras consecutivas.

Ejemplo: Para la frase "hola mundo", los n-gramas serían: ["hola", "mundo", "hola mundo"]

In [182]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Crear el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')

# Aplicar TF-IDF a comentarios
# tfidf_comments = tfidf_vectorizer.fit_transform(model1['clean_text'])
tfidf_transcripts = tfidf_vectorizer.fit_transform(model1['clean_text'])

# Convertir a DataFrame para visualizar las características
# tfidf_comments_df = pd.DataFrame(tfidf_comments.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_transcripts_df = pd.DataFrame(tfidf_transcripts.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# print(f'tfidf_comments: {tfidf_comments_df.shape}')
print(f'tfidf_transcript: {tfidf_transcripts_df.shape}')


tfidf_transcript: (1496, 5000)


# Get recommendations

La similitud coseno (cosine similarity) es una medida de similitud entre dos vectores en un espacio vectorial que calcula el coseno del ángulo entre ellos. Es ampliamente utilizada en procesamiento de lenguaje natural (NLP), recuperación de información, y otras áreas relacionadas con la comparación de texto o documentos, ya que es eficiente y proporciona una buena estimación de la similitud entre documentos o textos representados como vectores.

In [172]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity([[1, 1, 3], [1, 2, 4]], [[1, 1, 5], [1, 3, 5]])
similarity.flatten()

array([0.98644005, 0.96832966, 0.96590921, 0.99591   ])

In [184]:
from sklearn.metrics.pairwise import cosine_similarity

# Suponiendo que conoces el video_id del video para el cual quieres recomendaciones
video_id = model1['video_id'][50]  # Reemplaza con el video_id del video específico

# Extraer el índice del video en el DataFrame
index = model1[model1['video_id'] == video_id].index[0]

# Calcular la similitud del coseno entre este video y todos los demás
cosine_similarities = cosine_similarity(tfidf_transcripts[index], tfidf_transcripts).flatten()

# Crear un DataFrame con los video_ids y las similitudes correspondientes
similar_videos = pd.DataFrame({
    'video_id': model1['video_id'],
    'similarity': cosine_similarities
})

# Ordenar los videos por similitud, excluyendo el propio video
similar_videos = similar_videos[similar_videos['video_id'] != video_id].sort_values(by='similarity', ascending=False)

similar_videos = similar_videos.merge(df_videos[['videoId', 'title']], left_on='video_id', right_on='videoId')
similar_videos.drop(columns='videoId', inplace=True)

# Mostrar los 5 videos más similares
video_title = df_videos[df_videos['videoId'] == video_id]['title'].iloc[0]
print(f'video watched: {video_title} [{video_id}]\n')
print(f'videos recommended: {similar_videos.head()}')

video watched: AI in Transportation From Autonomous Vehicles to Traffic Management #shorts #ai [0CqyPmLw8X4]

videos recommended:       video_id  similarity                                              title
0  v6OB80Vt1Dk    1.000000     5 Mind-blowing Artificial Intelligence Tools 🤯
1  P_TAklE-UmE    0.360151  Kittl AI Unveiled: The Unstoppable AI Vector G...
2  Cu-UBInM2eo    0.330453  Create Stunning Animated Explainer Videos in M...
3  EL2woThmPWM    0.310712             $0,00 AI Tools That Can Make You Money
4  4p00K5_N7dI    0.298531  🔥 Top 10 Mind Blowing AI Tools For 2023 | 10 B...
